# For reprodutibility, we used the following 'versions' of the code and datasets
- MINOTAUR commit used: https://github.com/Mirandatz/minotaur/tree/6a045d5ec7de720804f2d820dae701ca1420235b
- DATASET commit used: https://github.com/Mirandatz/minotaur.datasets/tree/7c845a28274ccf858097ddd1578fd9624aa957d5

In [1]:
import subprocess
from pathlib import Path

import pandas as pd

In [2]:
MINOTAUR_LINE_SEP = '==============================================================================='
MINOTAUR_PATH =  Path('c:/') / 'source' / 'minotaur' / 'minotaur' / 'minotaur' / 'bin' / 'x64' / 'release' / 'netcoreapp3.0' / 'Minotaur.exe'
DATASET_DIR =  Path('c:/') / 'source' / 'minotaur.datasets' / 'yeast' / '2-ready-for-minotaur'
OUTPUT_DIR = Path.cwd() / 'yeast-output'
FOLD_COUNT = 10

In [3]:
# Create the output directories, if necessary
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
        
for fold_nr in range(FOLD_COUNT):
    fold_output_dir = OUTPUT_DIR / f'fold-{fold_nr}'
    fold_output_dir.mkdir(parents=True, exist_ok=True)

#### In the next cells we run MINOTAUR for each fold of the dataset.

The following command line arguments are mandatory; they are paths to csv files:
- `--train-data`
- `--train-labels`
- `--test-data`
- `--test-labels`

The train files are used to guide the evolutionary process.
The test files are used to compute the final population's fitnesses.

The `--output-directory` argument defines a path for MINOTAUR to create its output files, `final-population-fitnesses.txt`, containing the fitnesses of the individuals from the final population, and `final-population-individuals.txt`, containing the individuals from the final population. Both files are stored in a human readable format; each individual is separated by a line of with containing only "=" symbols.

The `--classification-type` argument defines how the dataset labels should be parsed and how the consequent of rules should be generated / modified. It also defines how some metrics (like FScore) are computed.

The remaining arguments are self-explanatory

In [4]:
def run_minotaur(formated_args: str):
    subprocess_args = str(MINOTAUR_PATH) + ' ' +  formated_args
    subprocess.call(subprocess_args)

In [ ]:
%%time
dfs = []
for fold_nr in range(FOLD_COUNT):
    args = '--train-data=' + str(DATASET_DIR / f'fold-{fold_nr}' / 'train-data.csv')
    args += ' --train-labels=' + str(DATASET_DIR / f'fold-{fold_nr}' / 'train-labels.csv')
    args += ' --test-data=' + str(DATASET_DIR / f'fold-{fold_nr}' / 'test-data.csv')
    args += ' --test-labels=' + str(DATASET_DIR / f'fold-{fold_nr}' / 'test-labels.csv')
    args += ' --output-directory=' + str(OUTPUT_DIR / f'fold-{fold_nr}')
    args += ' --classification-type=multilabel'
    args += ' --fitness-metrics=fscore'
    args += ' --fitness-metrics=average-rule-volume'
    args += ' --max-generations=100'
    args += ' --population-size=50'
    args += ' --mutants-per-generation=100'
    args += ' --cache-size-fitnesses=200'
    run_minotaur(args)

#### In the next cells we load, parse and post-process the outputs of MINOTAUR 
One of MINOTAUR's current implementation limitation is that it assumes that all objectives must be maximized, thus, in order to optimize the `rule-count` metric, we must compute is as negative value during the MINOTAUR's execution.
In order to make `rule-count` more readable, after we parse it we multiply it by -1 to obtain the correct value.

In [ ]:
def parse_fitnesses(path: Path) -> pd.DataFrame:
    # The output of the algorithm, including the fitnesses,
    # are stored in a human-readable format, so we must do some 
    # work to parse it
    lines = path.read_text().split('\n')
        
    # Removing 'line separators'
    lines = [l for l in lines if MINOTAUR_LINE_SEP not in l]
    
    # Removing the endline, '[' and ']' characters
    lines = [l[:-1].replace('[','').replace(']','').split(',') for l in lines]
    
    # Creating the dataframe, fixing the column names and data types
    df = pd.DataFrame(lines)
    df = df.rename(columns={0: 'fscore', 1:'avg-rule-vol'})
    df = df.astype({'fscore': 'float', 'avg-rule-vol':'float'})
    return df

In [ ]:
for fold_nr in range(FOLD_COUNT):
    fold_path = OUTPUT_DIR / f'fold-{fold_nr}' / 'final-population-fitnesses.txt'
    df = parse_fitnesses(fold_path)
    df['fold'] = fold_nr
    dfs.append(df)
df = pd.concat(dfs)

In [ ]:
# Lets take a peek of the dataframe, Each line represents a given individuals fitness in a given fold.
df.head()

In [ ]:
# Here we are select the best individual of each fold, with 'best' meaning highest f-score
distinct_values = df.sort_values(by='fscore', ascending=False).drop_duplicates(['fold', 'fscore'])
indices_of_best = distinct_values.groupby('fold')['fscore'].transform(max) == distinct_values['fscore']
best_individuals = distinct_values[indices_of_best].sort_values(by='fold')

In [ ]:
best_individuals

In [ ]:
best_individuals.drop(columns=['fold', 'avg-rule-vol']).mean()